In [701]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from xgboost import XGBRegressor

In [702]:
df=pd.read_csv('/workspaces/Laptop_Price_Predictor/data/raw/laptop_data.csv')

In [703]:
df.head()

,Unnamed: 0,Brand,Name,Price,Processor_Name,Processor_Brand,RAM_Expandable,RAM,RAM_TYPE,Ghz,Display_type,Display,GPU,GPU_Brand,SSD,HDD,Adapter,Battery_Life
0,0,HP,HP Chromebook 11A-NA0002MU (2E4N0PA) Laptop (1...,22990,MediaTek Octa-core,MediaTek,Not Expandable,4 GB,DDR4 RAM,2.0 Ghz Processor,LED,11.6,Integrated Graphics,MediaTek,64 GB SSD Storage,No HDD,45,Upto 12 Hrs Battery Life
1,1,Lenovo,Lenovo Ideapad Slim 3 (82KU017KIN) Laptop (15....,36289,AMD Hexa-Core Ryzen 5,AMD,12 GB Expandable,8 GB,DDR4 RAM,4.0 Ghz Processor,LCD,15.6,Radeon,AMD,512 GB SSD Storage,No HDD,65,Upto 11 Hrs Battery Life
2,3,Dell,Dell G15-5520 (D560822WIN9B) Laptop (15.6 Inch...,78500,Intel Core i5 (12th Gen),Intel,32 GB Expandable,16 GB,DDR5 RAM,3.3 Ghz Processor,LCD,15.6,"GeForce RTX 3050 GPU, 4 GB",NVIDIA,512 GB SSD Storage,No HDD,56,Upto 10 Hrs Battery Life
3,4,HP,HP 15s-fy5007TU (91R03PA) Laptop (15.6 Inch | ...,55490,Intel Core i5 (12th Gen),Intel,8 GB Expandable,8 GB,DDR4 RAM,4.2 Ghz Processor,LCD,15.6,Iris Xe,Intel,512 GB SSD Storage,No HDD,no,Upto 7.30 Hrs Battery Life
4,6,Infinix,Infinix Inbook Y2 Plus XL29 Laptop (15.6 Inch ...,21990,Intel Core i3 (11th Gen),Intel,Not Expandable,8 GB LP,LPDDR4X RAM,1.7 Ghz Processor,LCD,15.6,UHD,Intel,512 GB SSD Storage,No HDD,45,Upto 8 Hrs Battery Life


In [704]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3976 entries, 0 to 3975
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       3976 non-null   int64 
 1   Brand            3976 non-null   object
 2   Name             3976 non-null   object
 3   Price            3976 non-null   int64 
 4   Processor_Name   3976 non-null   object
 5   Processor_Brand  3976 non-null   object
 6   RAM_Expandable   3976 non-null   object
 7   RAM              3976 non-null   object
 8   RAM_TYPE         3976 non-null   object
 9   Ghz              3976 non-null   object
 10  Display_type     3976 non-null   object
 11  Display          3976 non-null   object
 12  GPU              3968 non-null   object
 13  GPU_Brand        3972 non-null   object
 14  SSD              3976 non-null   object
 15  HDD              3976 non-null   object
 16  Adapter          3976 non-null   object
 17  Battery_Life     3558 non-null   

In [705]:
df.shape

(3976, 18)

In [706]:
df.dtypes

Unnamed: 0          int64
Brand              object
Name               object
Price               int64
Processor_Name     object
Processor_Brand    object
RAM_Expandable     object
RAM                object
RAM_TYPE           object
Ghz                object
Display_type       object
Display            object
GPU                object
GPU_Brand          object
SSD                object
HDD                object
Adapter            object
Battery_Life       object
dtype: object

In [707]:
df.isnull().sum()

Unnamed: 0           0
Brand                0
Name                 0
Price                0
Processor_Name       0
Processor_Brand      0
RAM_Expandable       0
RAM                  0
RAM_TYPE             0
Ghz                  0
Display_type         0
Display              0
GPU                  8
GPU_Brand            4
SSD                  0
HDD                  0
Adapter              0
Battery_Life       418
dtype: int64

In [708]:
df.describe()

,Unnamed: 0,Price
count,3976.000000,3976.000000
mean,2181.495724,72432.528672
std,1297.029657,52207.650948
min,0.000000,7990.000000
25%,1058.750000,39873.250000
50%,2098.500000,58990.000000
75%,3342.250000,84990.000000
max,4408.000000,503890.000000


In [709]:
df.describe(include='object')

,Brand,Name,Processor_Name,Processor_Brand,RAM_Expandable,RAM,RAM_TYPE,Ghz,Display_type,Display,GPU,GPU_Brand,SSD,HDD,Adapter,Battery_Life
count,3976,3976,3976,3976,3976,3976,3976,3976,3976,3976,3968,3972,3976,3976,3976,3558
unique,31,3941,125,19,10,20,19,31,2,34,300,11,19,8,68,191
top,ASUS,Lenovo IdeaPad Flex 5 14IRU8 (82Y00051IN) Lapt...,Intel Core i5 (11th Gen),Intel,Not Expandable,8 GB,DDR4 RAM,4.2 Ghz Processor,LCD,15.6,UHD,Intel,512 GB SSD Storage,No HDD,65,65W Adapter
freq,989,2,316,3009,2157,1608,1505,331,2034,2205,651,1939,1931,3014,1458,803


In [710]:
for col in df.columns:
  print(f"{col}:{df[col].nunique()} unique values")

Unnamed: 0:3976 unique values
Brand:31 unique values
Name:3941 unique values
Price:1799 unique values
Processor_Name:125 unique values
Processor_Brand:19 unique values
RAM_Expandable:10 unique values
RAM:20 unique values
RAM_TYPE:19 unique values
Ghz:31 unique values
Display_type:2 unique values
Display:34 unique values
GPU:300 unique values
GPU_Brand:11 unique values
SSD:19 unique values
HDD:8 unique values
Adapter:68 unique values
Battery_Life:191 unique values


## Brand

In [711]:
df['Brand'].value_counts()

Brand
ASUS         989
Lenovo       811
HP           803
Dell         420
Acer         322
MSI          316
Samsung       61
Apple         49
Infinix       39
Microsoft     34
AVITA         23
Gigabyte      17
LG            15
Honor         14
iball         11
Xiaomi        10
Fujitsu        8
Zebronics      8
Wings          4
Colorful       3
Realme         3
Micromax       3
Coconics       3
Tecno          2
Smartron       2
Huawei         1
Razer          1
VAIO           1
Lava           1
Nokia          1
RDP            1
Name: count, dtype: int64

In [712]:
def brd(text):
    B=['RDP','Nokia','VAIO','Lava','Huawei','Razer','Smartron','Tecno','Colorful','Micromax','Coconics','Realme','Wings','Zebronics','Fujitsu']
    if text in B:
        return 'other brand'
    else:
        return text
    

df['Brand']=df['Brand'].apply(brd)
df.info()    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3976 entries, 0 to 3975
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       3976 non-null   int64 
 1   Brand            3976 non-null   object
 2   Name             3976 non-null   object
 3   Price            3976 non-null   int64 
 4   Processor_Name   3976 non-null   object
 5   Processor_Brand  3976 non-null   object
 6   RAM_Expandable   3976 non-null   object
 7   RAM              3976 non-null   object
 8   RAM_TYPE         3976 non-null   object
 9   Ghz              3976 non-null   object
 10  Display_type     3976 non-null   object
 11  Display          3976 non-null   object
 12  GPU              3968 non-null   object
 13  GPU_Brand        3972 non-null   object
 14  SSD              3976 non-null   object
 15  HDD              3976 non-null   object
 16  Adapter          3976 non-null   object
 17  Battery_Life     3558 non-null   

## Price

In [713]:
df['Price'].isna().sum()

np.int64(0)

In [714]:
df['Price'] = df['Price'] * 3.5 

## Processor Name

In [715]:
df['Processor_Name'].value_counts()

Processor_Name
Intel Core i5 (11th Gen)           316
Intel Core i5 (12th Gen)           294
AMD Hexa-Core Ryzen 5              245
AMD Octa-Core Ryzen 7              239
Intel Core i3 (11th Gen)           189
                                  ... 
Microsoft SQ1 Processor              1
AMD Dual-Core A9 APU Processor       1
AMD Octa-Core Ryzen 9 Processor      1
1.9 Ghz Processor                    1
2.6 Ghz Processor                    1
Name: count, Length: 125, dtype: int64

In [716]:
def proc(text):
    if 'Intel' in text:
        if 'i3' in text or 'i5' in text or 'i7' in text or 'i9' in text:
            return " ".join(text.split()[:3])
        else:
            return 'Other Intel'
    elif 'AMD' in text:
        return " ".join(text.split()[:2])
    elif 'Apple' in text:
        return 'Apple'
    else:
        return 'Other Processor'

def gen(text):
    if 'Intel' in text:
        if 'i3' in text or 'i5' in text or 'i7' in text or 'i9' in text:
            if text.split()[3][-2:]=='th':
                return int(text.split('th')[0].split('(')[-1])
            elif text.split()[3][-2:]=='st':
                return int(text.split('st')[0].split('(')[-1])
            elif text.split()[3][-2:]=='nd':
                return int(text.split('nd')[0].split('(')[-1])
            elif text.split()[3][-2:]=='rd':
                return int(text.split('rd')[0].split('(')[-1])
            else:
                return 0
                
        else:
            return 0
    else:
        return 0

def os(text):
    if ('Windows 11' in text.split('|')[3]) or 'Windows 10' in text.split('|')[3] or ('Windows 8.1' in text.split('|')[3]) or ('DOS' in text.split('|')[3]) or ('Linux' in text.split('|')[3]) or ('Ubuntu' in text.split('|')[3]) :
        return text.split('|')[3].strip()
    elif 'macOS' in text.split('|')[3]:
        return 'macOS'
    else:
        return 'Others'
    
    
    
df['processor']=df['Processor_Name'].apply(proc)
df['generation']=df['Processor_Name'].apply(gen)  
df['generation']=df['generation'].astype(np.int16)
df['OpSys']=df['Name'].apply(os)

##df.drop(columns=['Name','Processor_Name'],inplace=True)

In [717]:
df['RAM'].value_counts()

RAM
8 GB          1608
16 GB         1088
4 GB           547
16 GB LP       299
8 GB LP        182
32 GB           88
32 GB LP        48
4 GB LP         36
2 GB            24
16 GB RAM       16
8 GB RAM        14
64 GB            6
12 GB            6
16 GB RAM        5
32 GB RAM        3
3 GB             2
18 GB RAM        1
36 GB RAM        1
24 GB            1
6 GB             1
Name: count, dtype: int64

In [718]:
df['SSD'].value_counts()

SSD
512 GB SSD Storage     1931
NO SSD                  664
1024 GB SSD Storage     591
256 GB SSD Storage      356
256 GB SSD Storage      205
128 GB SSD Storage       57
128 GB SSD Storage       48
2048 GB SSD Storage      34
32 GB SSD Storage        25
64 GB SSD Storage        23
512 GB SSD Storage       16
16 GB SSD Storage        12
8 GB SSD Storage          5
512  SSD Storage          2
16 GB SSD Storage         2
256  SSD Storage          2
1 GB SSD Storage          1
4098 GB SSD Storage       1
32 GB SSD Storage         1
Name: count, dtype: int64

## Display

In [719]:
df['Display'].unique()

array(['11.6 ', '15.6 ', '14 ', '16 ', '17.3 ', '13.3 ', '16.1 ', '13.6 ',
       '14.2 ', '14.1 ', '15.3 ', '13.4 ', '15 ', '10.5 ', '13.5 ', '13 ',
       '12.4 ', '16.2 ', 'OLED Display With Touchscreen', '14.0 ', '17 ',
       '18 ', '14.5 ', '12 ', '10.1 ', '15. ', '14.9 ', '11 ', '12.3 ',
       '16.6 ', '12.2 ', '12.5 ', '15.4 ', '12.0 '], dtype=object)

In [720]:
df[df['Display']=='OLED Display With Touchscreen']

,Unnamed: 0,Brand,Name,Price,Processor_Name,Processor_Brand,RAM_Expandable,RAM,RAM_TYPE,Ghz,...,Display,GPU,GPU_Brand,SSD,HDD,Adapter,Battery_Life,processor,generation,OpSys
774,817,HP,HP ENVY 15 ep1087TX (54B88PA) Laptop (| Core i...,662900.0,Intel Core i9 (11th Gen),Intel,Not Expandable,32 GB,DDR4 RAM,4.8 Ghz Processor,...,OLED Display With Touchscreen,"GeForce RTX 3060 GPU, 6 GB",NVIDIA,1024 GB SSD Storage,No HDD,83,Upto 7.15 Hrs Battery Life,Intel Core i9,11,Windows 11


In [721]:
df.drop(774,inplace=True)

In [722]:
df['Display']=df['Display'].astype(np.float32)

In [723]:
df=df.drop(columns=['Unnamed: 0'])

In [724]:
def display_category(x):
    if x <= 14.0:
        return 'Small'
    elif x <= 15.6:
        return 'Medium'
    else:
        return 'Large'

df['Display_Category'] = df['Display'].apply(display_category)
df['Display_Category'].value_counts()


Display_Category
Large     2622
Small     1293
Medium      60
Name: count, dtype: int64

In [725]:
df['Battery_Life'].unique()

array([' Upto 12 Hrs Battery Life', ' Upto 11 Hrs Battery Life',
       ' Upto 10 Hrs Battery Life', 'Upto 7.30 Hrs Battery Life',
       ' Upto 8 Hrs Battery Life', '45W Adapter',
       ' Upto 9 Hrs Battery Life', '150W Adapter',
       ' Upto 9.5 Hrs Battery Life', ' Upto 7 Hrs Battery Life',
       '230W Adapter', ' Upto 9.45 Hrs Battery Life',
       'Upto 7 Hrs Battery Life', ' Upto 6 Hrs Battery Life',
       ' Upto 17.5 Hrs Battery Life', '135W Adapter',
       'Upto 6.5 Hrs Battery Life', nan, '65W Adapter',
       'Upto 5 Hrs Battery Life', 'Upto 10 Hrs Battery Life',
       ' Upto 4 Hrs Battery Life', ' Upto 6.5 Hrs Battery Life',
       ' Upto 5 Hrs Battery Life', '90W Adapter',
       ' Upto 21 Hrs Battery Life', 'Upto 6 Hrs Battery Life',
       ' Upto 14.7 Hrs Battery Life', ' Upto 13 Hrs Battery Life',
       '52.5W Adapter', '180W Adapter', '41W Adapter', '45 / 65W Adapter',
       ' Upto 15 Hrs Battery Life', '120W Adapter',
       ' Upto 18 Hrs Battery Life', ' Upto 

In [726]:
df['Adapter'].value_counts()

Adapter
65     1458
no      777
45      463
41      156
150      91
       ... 
97        1
15        1
47        1
26        1
69        1
Name: count, Length: 68, dtype: int64

## Adapter

In [727]:
def adap(num):
    if num.isdigit()==True:
        return int(num)
    else:
        return 0

df['adapter']=df['Adapter'].apply(adap)
df.drop(columns=['Adapter'],inplace=True)

## GPU

In [728]:
df.dropna(subset=['GPU'], inplace=True)
print(f"Unique Values: {df['GPU'].unique()}\n")
print(f"Value count for each unique processor type: {df['GPU'].value_counts()}")

Unique Values: ['Integrated Graphics' 'Radeon' 'GeForce RTX 3050 GPU, 4 GB' 'Iris Xe'
 'UHD' 'UHD 620' 'GeForce RTX 2050 GPU, 4 GB' 'GeForce RTX 4050 GPU, 6 GB'
 'Arc' 'GeForce GTX 1650 GPU, 4 GB' 'UHD 605' 'GeForce RTX 3060 GPU, 6 GB'
 'GeForce GTX 1650 Max Q GPU, 4 GB' 'GeForce RTX 3050 Ti GPU, 4 GB'
 'Integrated' 'Geforce GTX 1650 GPU, 4 GB' 'Radeon Vega 3' 'UHD Graphics'
 'UHD GPU, 128 MB' 'M1' 'HD 6000' 'UHD 600' 'Radeon Vega 6'
 'M2 Integrated' 'HD 620' 'Geforce MX150 GPU, 2 GB' 'Radeon Vega 8'
 'Geforce GTX 1650 Max-Q GPU, 4 GB' 'GeForce GTX 1650 Ti GPU, 4 GB'
 'Radeon RX Vega 10' 'HD 520' 'Radeon RX 5500M GPU, 4 GB'
 'GeForce GTX 3050 GPU, 4 GB' 'Radeon 610M' 'Radeon RX6500M GPU, 4 GB'
 'GeForce GTX 1650 MAX Q GPU, 4 GB' 'GeForce RTX 2050' 'HD 500'
 'Radeon Graphics' 'GeForce RTX 3070 Ti GPU, 8 GB'
 'GeForce GTX 1050 GPU, 4 GB' 'Geforce RTX 3050 GPU, 4 GB' 'M2'
 'Radeon RX 6500M GPU, 4 GB' 'Radeon RX 5600M GPU, 6 GB'
 'GeForce MX450 GPU, 2 GB' 'GeForce RTX 4060 GPU, 8 GB' 'Adre

In [729]:
def categorize_gpu_tier(gpu):
    if pd.isna(gpu):
        return 'Unknown'
    
    gpu = str(gpu).lower()

    # Entry-level (Integrated / Basic)
    if any(x in gpu for x in [
        'uhd', 'hd', 'integrated', 'r2', 'r3', 'r4', 'r5', 'r7', 
        '520', '530', '535', '610m', 'vega 3', 'vega 2', 'intel hd graphics',
        'vega 3', 'vega 6', 'vega 8', 'graphics', 'iris', '820m', 
        '830m', '920m', '930m', '940m', 'gt 740m', 'gt 820m', 'uma'
    
    ]):
        return 'Entry-level'

    # Low-end (Budget Discrete)
    elif any(x in gpu for x in [
        'mx', 'arc', 'vega 5', 'vega 6', 'vega 7', 'vega 8',
        'vega 10', 'rx640', 'rx 640', 'rx 5500m', 'rx5500', 'rx 5500', 'rx vega',
        'iris xe', 'iris plus', 'iris pro', 'rx 640 gpu', 'rx 640m', 'rx 640 ', 'rx640 ', 'iris x', 'radeon'
    ]):
        return 'Low-end'

    # Mid-end (Mainstream Gaming / Productivity)
    elif any(x in gpu for x in [
        'gtx 1050', 'gtx 1060', 'gtx 1650', 'gtx 1660',
        'rtx 2050', 'rtx 3050', 'rtx 3050 ti', 'rtx 3060',
        't550', 't600', 't1200', 'p620', 'a1000',
        'rx 5600', 'rx 6500', 'rx 6600', 'rx 6650', 'rx 6700s',
        '680m', '760m', '780m', 'iris xe max', 'gtx 850m', 'gtx 960m', 'gtx 1070', 'gtx 3050', 'rtx 1650', 
    'rx 560x', 'rx6500', 'rx6550', 'rx6700', 'pro 555x'
    ]):
        return 'Mid-end'

    # High-end (Performance / Workstation / Enthusiast)
    elif any(x in gpu for x in [
        'rtx 2060', 'rtx 2070', 'rtx 2080', 'rtx 3050 ti', 'rtx 3060',
        'rtx 3070', 'rtx 3070 ti', 'rtx 3080', 'rtx 3080 ti',
        'rtx 4060', 'rtx 4070', 'rtx 4080', 'rtx 4090',
        'rtx a2000', 'rtx a500', 'rtx a1000',
        'quadro t2000', 'quadro t600', 'quadro t550',
        'rx 6700m', 'rx 6700s', 'rx 6800m', 'rx 7600s',
        'rx 6650m', 'rx 580', 'pro 5500m',
        'm2 max', 'm3 max','rtx 4050', 'radeon pro 450', 'radeon gpu, 6', 'm2', 'm1'
    ]):
        return 'High-end'

    else:
        return 'Unknown'


df['GPU_Tier'] = df['GPU'].apply(categorize_gpu_tier)
print(df['GPU_Tier'].value_counts())


GPU_Tier
Entry-level    2180
Mid-end         822
Low-end         611
High-end        340
Unknown          14
Name: count, dtype: int64


In [730]:
df.drop(df[df['GPU_Tier']=='Unknown'].index, inplace=True)

## GPU Brand

In [731]:
print(f"Unique Values: {df['GPU_Brand'].unique()}\n")
print(f"Value count for each unique GPU_Brand type: {df['GPU_Brand'].value_counts()}\n")

Unique Values: ['MediaTek' 'AMD' 'NVIDIA' 'Intel' 'Apple' 'NIVIDIA' 'Nvidia' nan 'ATI']

Value count for each unique GPU_Brand type: GPU_Brand
Intel       1931
NVIDIA      1370
AMD          616
Apple         19
Nvidia         8
MediaTek       3
NIVIDIA        1
ATI            1
Name: count, dtype: int64



In [732]:
df['GPU_Brand'].fillna(df['GPU_Brand'].value_counts().index[0],inplace=True)

/tmp/ipykernel_1070/3459734782.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['GPU_Brand'].fillna(df['GPU_Brand'].value_counts().index[0],inplace=True)


In [733]:
df.drop(df[df['GPU_Brand']=='ATI'].index, inplace=True)
df['GPU_Brand'] = df['GPU_Brand'].astype('category')

def clean_gpu_brand(value):
    if 'Intel' in value:
        return 'Intel'
    elif 'NVIDIA' in value or 'Nvidia' in value or 'NIVIDIA' in value:
        return 'NVIDIA'
    elif 'AMD' in value:
        return 'AMD'
    elif 'Apple' in value:
        return 'Apple'
    else:
        return 'Other'

df['GPU_Brand'] = df['GPU_Brand'].apply(clean_gpu_brand)

## Ram Type

In [734]:
df['RAM_TYPE'] = df['RAM_TYPE'].str.strip().str.upper()
mapping = {
    'DDR4 RAM': 'DDR4',
    'DDR5 RAM': 'DDR5',
    'DDR3 RAM': 'DDR3',
    'DDR2 RAM': 'DDR2',
    'LPDDR3 RAM' : 'LPDDR3',
    'LPDDR4 RAM': 'LPDDR4',
    'LPDDR4X RAM': 'LPDDR4X',
    'LPDDR5 RAM': 'LPDDR5',
    'LPDDR5X RAM': 'LPDDR5X'
}
df['RAM_TYPE'] = df['RAM_TYPE'].replace(mapping)
df['RAM_TYPE'] = df['RAM_TYPE'].astype('category')


In [735]:
df.head()

,Brand,Name,Price,Processor_Name,Processor_Brand,RAM_Expandable,RAM,RAM_TYPE,Ghz,Display_type,...,GPU_Brand,SSD,HDD,Battery_Life,processor,generation,OpSys,Display_Category,adapter,GPU_Tier
0,HP,HP Chromebook 11A-NA0002MU (2E4N0PA) Laptop (1...,80465.0,MediaTek Octa-core,MediaTek,Not Expandable,4 GB,DDR4,2.0 Ghz Processor,LED,...,Other,64 GB SSD Storage,No HDD,Upto 12 Hrs Battery Life,Other Processor,0,Others,Small,45,Entry-level
1,Lenovo,Lenovo Ideapad Slim 3 (82KU017KIN) Laptop (15....,127011.5,AMD Hexa-Core Ryzen 5,AMD,12 GB Expandable,8 GB,DDR4,4.0 Ghz Processor,LCD,...,AMD,512 GB SSD Storage,No HDD,Upto 11 Hrs Battery Life,AMD Hexa-Core,0,Windows 11,Large,65,Low-end
2,Dell,Dell G15-5520 (D560822WIN9B) Laptop (15.6 Inch...,274750.0,Intel Core i5 (12th Gen),Intel,32 GB Expandable,16 GB,DDR5,3.3 Ghz Processor,LCD,...,NVIDIA,512 GB SSD Storage,No HDD,Upto 10 Hrs Battery Life,Intel Core i5,12,Windows 11,Large,56,Mid-end
3,HP,HP 15s-fy5007TU (91R03PA) Laptop (15.6 Inch | ...,194215.0,Intel Core i5 (12th Gen),Intel,8 GB Expandable,8 GB,DDR4,4.2 Ghz Processor,LCD,...,Intel,512 GB SSD Storage,No HDD,Upto 7.30 Hrs Battery Life,Intel Core i5,12,Windows 11,Large,0,Entry-level
4,Infinix,Infinix Inbook Y2 Plus XL29 Laptop (15.6 Inch ...,76965.0,Intel Core i3 (11th Gen),Intel,Not Expandable,8 GB LP,LPDDR4X,1.7 Ghz Processor,LCD,...,Intel,512 GB SSD Storage,No HDD,Upto 8 Hrs Battery Life,Intel Core i3,11,Windows 11,Large,45,Entry-level


## RAM

In [736]:
df['RAM'] = df['RAM'].astype(str).str.replace(r'[^0-9.]', '', regex=True)
df['RAM'] = pd.to_numeric(df['RAM'], errors='coerce')

In [737]:

df.rename(columns={'RAM': 'RAM_Size (GB)'}, inplace=True)
print(f"Unique Values: {df['RAM_Size (GB)'].unique()}\n")

Unique Values: [ 4  8 16 32 64 12  2 24  6  3]



In [738]:
standard_ram_sizes = [4, 8, 16, 32]
df = df[df["RAM_Size (GB)"].isin(standard_ram_sizes)].reset_index(drop=True)

print(df["RAM_Size (GB)"].value_counts())

RAM_Size (GB)
8     1794
16    1402
4      581
32     137
Name: count, dtype: int64


## Processor Speed

In [739]:
df['Ghz'] = df['Ghz'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
df.rename(columns={'Ghz': 'Processer_speed'}, inplace=True)

In [740]:
print(f"Unique Values: {df['Processer_speed'].unique()}\n")

Unique Values: [2.  4.  3.3 4.2 1.7 0.  2.5 2.2 2.9 3.4 2.6 3.  4.7 3.1 2.7 2.8 2.4 2.1
 4.8 1.2 1.  1.8 1.1 2.3 1.6 0.8 1.3 1.5 1.9 5.5 1.4]



## SSD

In [741]:
df['SSD'] = df['SSD'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)

In [742]:
df.rename(columns={'SSD': 'SSD(GB)'}, inplace=True)

In [743]:
standard_sizes = [0, 128, 256, 512, 1024]

# Keep only these values
df = df[df['SSD(GB)'].isin(standard_sizes)]

# Check results
print(df['SSD(GB)'].value_counts())

SSD(GB)
512.0     1935
1024.0     586
256.0      562
128.0      101
Name: count, dtype: int64


In [744]:
df['SSD(GB)'] = df['SSD(GB)'].fillna(0)

## HDD

In [745]:
def convert_to_gb(hdd_str):
    if pd.isna(hdd_str) or 'No HDD' in str(hdd_str):
        return 0
    size_unit = re.search(r'(\d+)\s*(GB|TB)', str(hdd_str))
    if size_unit:
        size, unit = float(size_unit.group(1)), size_unit.group(2)
        return size * 1024 if unit == 'TB' else size
    return 0  
df['HDD'] = df['HDD'].apply(convert_to_gb).astype(int)

In [746]:
df.rename(columns={'HDD': 'HDD(GB)'}, inplace=True)
print(f"Unique Values: {df['HDD(GB)'].unique()}\n")

Unique Values: [   0 1024 2048]



In [747]:
standard_hdd = [0, 500, 1024]

# Keep only these values
df = df[df['HDD(GB)'].isin(standard_hdd)]

# Check results
print(df['HDD(GB)'].value_counts())

HDD(GB)
0       2907
1024     276
Name: count, dtype: int64


## RAM Expandable

In [748]:
if 'RAM_Expandable' in df.columns:
    
    df['RAM_Expandable'] = df['RAM_Expandable'].astype(str).str.replace(r'[^0-9.]', '', regex=True)
    df['RAM_Expandable'] = pd.to_numeric(df['RAM_Expandable'], errors='coerce')


print("Unique Values:", df['RAM_Expandable'].dropna().unique())

Unique Values: [12. 32.  8. 16.  4. 64. 48.  2.]


In [749]:
df['RAM_Expandable'] = df['RAM_Expandable'].apply(lambda x: 'Yes' if pd.notna(x) else 'No')

print(df['RAM_Expandable'].value_counts())


RAM_Expandable
No     1715
Yes    1468
Name: count, dtype: int64


In [750]:
df.head

<bound method NDFrame.head of         Brand                                               Name     Price  \
1      Lenovo  Lenovo Ideapad Slim 3 (82KU017KIN) Laptop (15....  127011.5   
2        Dell  Dell G15-5520 (D560822WIN9B) Laptop (15.6 Inch...  274750.0   
3          HP  HP 15s-fy5007TU (91R03PA) Laptop (15.6 Inch | ...  194215.0   
4     Infinix  Infinix Inbook Y2 Plus XL29 Laptop (15.6 Inch ...   76965.0   
5        Acer  Acer One 14 Z8-415 (UN.599SI.020) Laptop (14 I...  122465.0   
...       ...                                                ...       ...   
3909     ASUS  ASUS Zenbook UX430UN-GV022T Laptop (14 Inch | ...  297465.0   
3910     ASUS  ASUS Vivobook K15 OLED K513EA-L301WS Laptop (1...  140108.5   
3911     ASUS  ASUS VivoBook Pro 15 OLED M3500QC-L1711WS Lapt...  265965.0   
3912     ASUS  ASUS TUF FX505DT-AL118T Laptop (15.6 Inch | AM...  251681.5   
3913   Lenovo  Lenovo Legion 5i (82AU00CFIN) Laptop (15.6 Inc...  244965.0   

                          Process

## Processor Brand

In [751]:
print(f"Unique Values: {df['Processor_Brand'].unique()}\n")
print(f"Value count for each unique processor type: {df['Processor_Brand'].value_counts()}")

Unique Values: ['AMD' 'Intel' 'Apple' '2.3' '3.4' 'MediaTek' '3.3' '1.6' '3.0' '2.6']

Value count for each unique processor type: Processor_Brand
Intel       2389
AMD          767
Apple         18
3.4            2
3.0            2
2.3            1
MediaTek       1
3.3            1
1.6            1
2.6            1
Name: count, dtype: int64


In [752]:
valid_brands = ["AMD", "Apple", "Intel"]

df = df[df["Processor_Brand"].isin(valid_brands)].reset_index(drop=True)
df.shape

(3174, 22)

In [753]:
def split_processor_details(value):
    if pd.isna(value) or str(value).strip() == "":
        return ("Unknown", "Unknown")
    
    text = value.strip()

    # Extract brand (first word before space)
    brand = text.split()[0]

    # Remove everything inside parentheses and after it
    text_before_parenthesis = re.split(r'\(', text)[0].strip()

    # Remove the brand name from the start
    series = re.sub(rf'^{brand}\s*', '', text_before_parenthesis, flags=re.IGNORECASE).strip()

    return (brand, series)

# Apply to DataFrame
df[["Processor_Brand_Extracted", "Processor_Series"]] = df["Processor_Name"].apply(lambda x: pd.Series(split_processor_details(x)))

In [754]:
import re

def categorize_processor(row):
    name = row['Processor_Series']
    brand = row['Processor_Brand']
    
    if brand == 'Intel':
        # Capture i3/i5/i7/i9 or Core Ultra, Core series, etc.
        match = re.search(r'(i[3579]|core\s*[3579])', name, re.I)
        if match:
            # Standardize naming: convert "core 5" → "i5"
            text = match.group(0).lower()
            if "core" in text:
                # Extract the number after "core"
                num = re.search(r'[3579]', text).group(0)
                return f"Intel Core I{num}"
            else:
                return f"Intel Core {text.upper()}"
        elif re.search(r'Ultra\s*5', name, re.I):
            return "Intel Core Ultra 5"
        elif re.search(r'Ultra\s*7', name, re.I):
            return "Intel Core Ultra 7"
        elif re.search(r'Ultra\s*9', name, re.I):
            return "Intel Core Ultra 9"
        elif re.search(r'Pentium', name, re.I):
            return "Intel Pentium"
        elif re.search(r'Celeron', name, re.I):
            return "Intel Celeron"
        else:
            return "Intel Other"
    
    elif brand == 'AMD':
        # Ryzen families or Athlon/A-series
        match = re.search(r'Ryzen\s*[3579]', name, re.I)
        if match:
            return f"AMD {match.group(0).title()}"
        elif re.search(r'Athlon', name, re.I):
            return "Athlon"
        elif re.search(r'A\d', name, re.I):
            # e.g., A6, A8, A10, A12
            a_match = re.search(r'A\d+', name, re.I)
            return "AMD A-Series"
        else:
            return "AMD Other"
    
    elif brand == 'Apple':
        # All M-series grouped together
        return "Apple M-series"
    
    else:
        return "Other"

# Apply to DataFrame
df['Processor_Series'] = df.apply(categorize_processor, axis=1)

# Quick look at the result
print(df['Processor_Series'].value_counts())


Processor_Series
Intel Core I5         1084
Intel Core I7          612
Intel Core I3          452
AMD Ryzen 5            341
AMD Ryzen 7            266
AMD Ryzen 3             91
Intel Core I9           68
Intel Celeron           58
Intel Core Ultra 7      49
AMD Ryzen 9             44
Intel Core Ultra 5      24
Intel Pentium           22
Apple M-series          18
Athlon                  17
Intel Core Ultra 9      14
AMD A-Series             8
Intel Other              6
Name: count, dtype: int64


In [755]:
def map_processor_performance(row):
    brand = row['Processor_Brand']
    series   = row['Processor_Series']

    if brand == 'Intel':
        if series in ['Intel Core Ultra 9', 'Intel Core I9']:
            return 'High-End'
        elif series in ['Intel Core I7',  'Intel Core Ultra 7']:
            return 'Upper Mid-End'
        elif series in ['Intel Core I5', 'Intel Core Ultra 5']:
            return 'Mid-End'
        elif series in ['Intel Core I3', 'Intel Celeron', 'Intel Pentium', 'Intel Other']:
            return 'Low-End'

    elif brand == 'AMD':
        if series in ['AMD Ryzen 9']:
            return 'High-End'
        elif series in ['AMD Ryzen 7']:
            return 'Upper Mid-End'
        elif series in ['AMD Ryzen 5']:
            return 'Mid-End'
        elif series in [
            'AMD Ryzen 3', 'Athlon', 'AMD A-Series', 'AMD Other'
        ]:
            return 'Low-End'

    elif brand == 'Apple':
        return 'Apple M-series'  

    return 'Other'  

In [756]:
df['Processor_Tier'] = df.apply(map_processor_performance, axis=1)
print(df['Processor_Tier'].value_counts())

Processor_Tier
Mid-End           1449
Upper Mid-End      927
Low-End            654
High-End           126
Apple M-series      18
Name: count, dtype: int64


## Display Type

In [757]:
df['Display_type'] = df['Display_type'].astype('category')
print(f"Unique Values: {df['Display_type'].unique()}\n")

Unique Values: ['LCD', 'LED']
Categories (2, object): ['LCD', 'LED']



In [758]:
df.head()

,Brand,Name,Price,Processor_Name,Processor_Brand,RAM_Expandable,RAM_Size (GB),RAM_TYPE,Processer_speed,Display_type,...,Battery_Life,processor,generation,OpSys,Display_Category,adapter,GPU_Tier,Processor_Brand_Extracted,Processor_Series,Processor_Tier
0,Lenovo,Lenovo Ideapad Slim 3 (82KU017KIN) Laptop (15....,127011.5,AMD Hexa-Core Ryzen 5,AMD,Yes,8,DDR4,4.0,LCD,...,Upto 11 Hrs Battery Life,AMD Hexa-Core,0,Windows 11,Large,65,Low-end,AMD,AMD Ryzen 5,Mid-End
1,Dell,Dell G15-5520 (D560822WIN9B) Laptop (15.6 Inch...,274750.0,Intel Core i5 (12th Gen),Intel,Yes,16,DDR5,3.3,LCD,...,Upto 10 Hrs Battery Life,Intel Core i5,12,Windows 11,Large,56,Mid-end,Intel,Intel Core I5,Mid-End
2,HP,HP 15s-fy5007TU (91R03PA) Laptop (15.6 Inch | ...,194215.0,Intel Core i5 (12th Gen),Intel,Yes,8,DDR4,4.2,LCD,...,Upto 7.30 Hrs Battery Life,Intel Core i5,12,Windows 11,Large,0,Entry-level,Intel,Intel Core I5,Mid-End
3,Infinix,Infinix Inbook Y2 Plus XL29 Laptop (15.6 Inch ...,76965.0,Intel Core i3 (11th Gen),Intel,No,8,LPDDR4X,1.7,LCD,...,Upto 8 Hrs Battery Life,Intel Core i3,11,Windows 11,Large,45,Entry-level,Intel,Intel Core I3,Low-End
4,Acer,Acer One 14 Z8-415 (UN.599SI.020) Laptop (14 I...,122465.0,Intel Core i5 (11th Gen) Processor,Intel,Yes,16,DDR4,0.0,LCD,...,45W Adapter,Intel Core i5,11,Windows 11,Small,45,Entry-level,Intel,Intel Core I5,Mid-End


In [759]:
cols_to_drop = [
    'Name','Processor_Name',
    'adapter', 'Battery_Life', 'Processor_Brand_Extracted'
]
# Drop columns
df = df.drop(columns=cols_to_drop)
# Check the updated DataFrame
df.head()

,Brand,Price,Processor_Brand,RAM_Expandable,RAM_Size (GB),RAM_TYPE,Processer_speed,Display_type,Display,GPU,GPU_Brand,SSD(GB),HDD(GB),processor,generation,OpSys,Display_Category,GPU_Tier,Processor_Series,Processor_Tier
0,Lenovo,127011.5,AMD,Yes,8,DDR4,4.0,LCD,15.6,Radeon,AMD,512.0,0,AMD Hexa-Core,0,Windows 11,Large,Low-end,AMD Ryzen 5,Mid-End
1,Dell,274750.0,Intel,Yes,16,DDR5,3.3,LCD,15.6,"GeForce RTX 3050 GPU, 4 GB",NVIDIA,512.0,0,Intel Core i5,12,Windows 11,Large,Mid-end,Intel Core I5,Mid-End
2,HP,194215.0,Intel,Yes,8,DDR4,4.2,LCD,15.6,Iris Xe,Intel,512.0,0,Intel Core i5,12,Windows 11,Large,Entry-level,Intel Core I5,Mid-End
3,Infinix,76965.0,Intel,No,8,LPDDR4X,1.7,LCD,15.6,UHD,Intel,512.0,0,Intel Core i3,11,Windows 11,Large,Entry-level,Intel Core I3,Low-End
4,Acer,122465.0,Intel,Yes,16,DDR4,0.0,LCD,14.0,Iris Xe,Intel,512.0,0,Intel Core i5,11,Windows 11,Small,Entry-level,Intel Core I5,Mid-End


In [760]:
df.to_csv('cleaned_laptop_dataset.csv', index=False)

In [761]:
encoder = LabelEncoder()
for col in ['Brand', 'Processor_Brand', 'RAM_TYPE', 'Display_type', 'OpSys', 'GPU','GPU_Brand' ,'processor']:
    df[col] = encoder.fit_transform(df[col])


In [762]:
df['RAM_Expandable'] = df['RAM_Expandable'].map({'Yes': 1, 'No': 0})
df['Display_Category'] = df['Display_Category'].map({'Large': 1, 'Small': 0})

In [763]:
tier_map = {'Low-end': 1, 'Entry-level': 2, 'Mid-end': 3, 'High-end': 4}
df['GPU_Tier'] = df['GPU_Tier'].map(tier_map)
df['Processor_Tier'] = df['Processor_Tier'].map(tier_map)

In [764]:
df.drop(columns=['Processor_Brand_Extracted', 'Processor_Series'], inplace=True, errors='ignore')

In [765]:
X = df.drop(columns=['Price'])
y = df['Price']

In [766]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [767]:
df.dtypes[df.dtypes == 'object']


Series([], dtype: object)

In [768]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(
    n_estimators=500,     # number of trees
    learning_rate=0.05,   # shrinkage step
    max_depth=8,          # tree depth
    subsample=0.8,        # row sampling
    colsample_bytree=0.8, # feature sampling
    random_state=42,
    objective='reg:squarederror'
)

xgb_model.fit(X_train, y_train)


,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [769]:
y_pred = xgb_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("R² Score:", r2)

RMSE: 66856.42321895038
R² Score: 0.7946298149490231


In [770]:
y_log = np.log1p(y)   # log(Price + 1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_log, test_size=0.2, random_state=42)

In [771]:
xgb = XGBRegressor(
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1
)

In [772]:
param_dist = {
    "n_estimators": [300, 500, 800, 1200],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [4, 6, 8, 10],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "min_child_weight": [1, 3, 5],
    "gamma": [0, 0.1, 0.3]
}
search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=20,              # try 20 random combinations
    scoring="neg_root_mean_squared_error",
    cv=3,                   # 3-fold cross validation
    verbose=1,
    random_state=42,
    n_jobs=-1
)

search.fit(X_train, y_train)

print("Best Parameters:", search.best_params_)


Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best Parameters: {'subsample': 0.8, 'n_estimators': 1200, 'min_child_weight': 3, 'max_depth': 10, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree': 0.6}


In [773]:
best_model = search.best_estimator_

y_pred_log = best_model.predict(X_test)
y_pred = np.expm1(y_pred_log)   # inverse log1p
y_true = np.expm1(y_test)

mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)   # take square root manually
r2 = r2_score(y_true, y_pred)

print("RMSE:", rmse)
print("R² Score:", r2)

RMSE: 64704.82583933683
R² Score: 0.8076357060244832
